In [1]:
!pip install datasets
!pip install librosa

In [52]:
import IPython.display as pds
import matplotlib.pyplot as plt
import numpy as np
import librosa
import warnings
import random
import tensorflow as tf
warnings.filterwarnings("ignore")

from datasets import load_dataset
from tensorflow.keras import layers

In [4]:
train_data, test_data = load_dataset('common_voice', 'id', split=['train', 'test'])

Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/id/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
train_audios = train_data['audio']
train_labels = train_data['sentence']

test_audios = test_data['audio']
test_labels = test_data['sentence']

In [76]:
def prepare_dataset(audios, labels):
  data_dict = dict()
  audio_array_list = list()
  label_list = list()

  for audio, label in zip(audios, labels):
    audio_array_list.append(audio['array'])
    label_list.append(label)
    sample_rate = audio['sampling_rate']

    data_dict['audio_array'], data_dict['labels'], data_dict['sample_rate'] = (audio_array_list, 
                                                                        label_list, 
                                                                        sample_rate)
  return data_dict

In [77]:
train_dataset = prepare_dataset(train_audios, train_labels)
test_dataset = prepare_dataset(test_audios, train_labels)

In [98]:
random_idx = random.randint(0, 1000)
print(train_dataset['audio_array'])

[array([ 0.        ,  0.        ,  0.        , ...,  0.00033009,
       -0.00013936, -0.00057721], dtype=float32), array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -5.9604645e-08,  1.3709068e-06, -5.1259995e-06], dtype=float32), array([ 0.        ,  0.        ,  0.        , ..., -0.00033897,
       -0.00042212, -0.00044096], dtype=float32), array([ 0.        ,  0.        ,  0.        , ..., -0.0009914 ,
       -0.00104195, -0.0010311 ], dtype=float32), array([ 0.        ,  0.        ,  0.        , ..., -0.00017935,
       -0.00025219, -0.00034171], dtype=float32), array([0.        , 0.        , 0.        , ..., 0.00078976, 0.0007484 ,
       0.0006696 ], dtype=float32), array([0.        , 0.        , 0.        , ..., 0.00026655, 0.00023812,
       0.00025833], dtype=float32), array([ 0.        ,  0.        ,  0.        , ..., -0.00046879,
       -0.00033593, -0.00015312], dtype=float32), array([0.        , 0.        , 0.        , ..., 0.00047004, 0.00031799,
       0.

In [82]:
pds.Audio(train_dataset['audio_array'][random_idx], rate=train_dataset['sample_rate'])

In [95]:
def create_model(n_class):
  model = tf.keras.Sequential()
  model.add(layers.Conv1D(32, kernel_size=(3, 3), activation='relu', input_shape=(1,)))
  model.add(layers.BatchNormalization())

  model.add(layers.Conv1D(48, kernel_size=(3, 3), activation='relu'))
  model.add(layers.BatchNormalization())

  model.add(layers.Conv1D(120, kernel_size=(3, 3), activation='relu'))
  model.add(layers.BatchNormalization())

  model.add(layers.MaxPooling1D(2, 2))
  model.add(layers.Dropout(0.5))

  model.add(layers.Flatten())

  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(n_class, activation='softmax'))

  return model

In [96]:
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
loss = tf.keras.losses.sparse_categorical_crossentropy

model = create_model(len(train_dataset))

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])

ValueError: ignored

In [87]:
x_train = train_dataset['audio_array']
y_train = train_dataset['labels']

x_test = test_dataset['audio_array']
y_test = test_dataset['labels']

epoch = 15

model.fit(x_train, 
          y_train, 
          epochs=epoch, 
          validation_data=(x_train, y_train), 
          batch_size=128)

NameError: ignored